In [35]:
import pandas as pd
import os
from prophet import Prophet
import numpy as np
import pickle
import boto3
import io
from io import BytesIO

In [ ]:
def load_data():
    directory= "/workspace/VoltWise/Data_Ingestion/daily_data/"

    # Read from an S3 bucket
    bucket_name = 'tempbucketvoltwise'
    key = 'CAL.csv'

    region_data = {}
    for filename in os.listdir(directory):
        if filename.endswith('.parquet'):
            # Extract the region name from the file name
            region = filename.split('.')[0]

            # Read the contents of the file into a variable
            filepath = os.path.join(directory, filename)
            df = pd.read_parquet(filepath)
            
            # Store the data in the dictionary with the region name as the key
            region_data[region] = df
    
    return region_data


In [2]:
# Specify the access keys
access_key_id = ''
secret_access_key = ''

# Create an S3 client
s3_client = boto3.client('s3', aws_access_key_id=access_key_id, aws_secret_access_key=secret_access_key)

In [15]:
# Read
response = s3_client.get_object(Bucket=bucket_name, Key=key)
data = response['Body'].read()
df = io.BytesIO(data)
df = pd.read_csv(df)

In [45]:
# Write
output_data = df.to_csv(index=False)

# Convert the CSV data to bytes
output_bytes = output_data.encode('utf-8')

# Write the CSV data to the bucket
s3_client.put_object(Body=output_bytes, Bucket=bucket_name, Key=key)

{'ResponseMetadata': {'RequestId': '3KZR52Y03YDWT0WW',
  'HostId': '1jFlKqvUS8+s2mX2Rsw/au4qZ1XMvrDhqZ4wbeU76wClYpONP+eukWnUO9+Rbt/CXtS8gLYk1tw=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '1jFlKqvUS8+s2mX2Rsw/au4qZ1XMvrDhqZ4wbeU76wClYpONP+eukWnUO9+Rbt/CXtS8gLYk1tw=',
   'x-amz-request-id': '3KZR52Y03YDWT0WW',
   'date': 'Thu, 29 Jun 2023 08:24:25 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"16ff39010b52de7b695c8e7f9e9235fe"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"16ff39010b52de7b695c8e7f9e9235fe"',
 'ServerSideEncryption': 'AES256'}

In [ ]:
region_data= load_data()            
regions = list(region_data.keys())
data_types = ['demand', 'generation']